In [1]:
import igraph as ig
import csv
from collections import defaultdict
import community
import networkx as nx
import numpy as np
import operator
import pydot
import plotly.plotly as py
import plotly.graph_objs as go
from networkx.drawing.nx_pydot import write_dot
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
import matplotlib
import plotly
plotly.tools.set_credentials_file(username='pcsplotly', api_key='ofJJAPLfVgSYzk4mmyzi')

filepath = "project_companies/"
louvainoutput = "louvain_clean_names.csv"
overlapgraph = "overlapgraph_clean"

# Loading the Louvain Output 
## and convert it into a dictionary of node:partition

In [2]:
##Building the main dictionary from the louvainoutput file
threshold = 200
belowthreshold = []
sizes = defaultdict()
partitioning = defaultdict()
print(filepath+louvainoutput)
with open(filepath+louvainoutput, "rt", encoding="utf8") as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    for row in reader:
        partition=row.pop(0)
        size=int(row.pop(0))
        if size < threshold:
                belowthreshold.append(partition)
        else:
            sizes[partition] = size
        for node in row:
            partitioning[node] = partition
           

project_companies/louvain_clean_names.csv


## Loading the overlapgraph as a proper nx graph

In [3]:
def buildWeightedGraph():
    print("Building weighted graph from %s..." % (filepath+overlapgraph))
    fh = open(filepath+overlapgraph, 'rb')
    G = nx.read_weighted_edgelist(fh, delimiter=";")
    print("Done.")
    return G

if True:
    original = buildWeightedGraph()

Building weighted graph from project_companies/overlapgraph_clean...
Done.


### check if overlapgraph and partitioning are consitent

In [4]:
membersinoverlap = list(map(int, original.nodes()))
uniqueinoverlap = list(set(membersinoverlap) - set(map(int,partitioning.keys())))
uniqueinclustrering = list(set(map(int,partitioning.keys())) - set(membersinoverlap))
bool(uniqueinclustrering) == bool(uniqueinoverlap) == False

True

## Build the induced graph based on the partitioning and the original graph

In [5]:
## get induced graph for all clusters
if False:
    induced = community.induced_graph(partitioning, original)
    # Save induced graph
    nx.write_gpickle(induced, filepath+"clustering_induced_graph.pkl")	
else:
    induced = nx.read_gpickle(filepath+"clustering_induced_graph.pkl")	

In [6]:
# removing too small clusters
induced.remove_nodes_from(belowthreshold)

In [7]:
sorted_sizes=[x for _,x in sorted(sizes.items(), key=lambda kv: kv[1], reverse=True)]
max_size = max(sorted_sizes)
min_size = min(sorted_sizes)

def normalize(value, max_size=max_size, min_size=min_size):
    return (value - min_size) / (max_size-min_size)
normalized_sizes = list(map(normalize,sorted_sizes))

In [8]:
# adding the size attribute
nx.set_node_attributes(induced, "size", sizes)

In [9]:
# size mapping <oldname><sizerank>  /ranking by size and relabelling
mapping = dict(zip([x for x,_ in sorted(sizes.items(), key=lambda kv: kv[1], reverse=True)], range(0, induced.number_of_nodes())))
induced_relabeled = nx.relabel_nodes(induced, mapping)

In [10]:
# export to .dot
nx.write_graphml(induced_relabeled,filepath+'induced.graphml') # Export NX graph to file
induced_ig = ig.read(filepath+'induced.graphml',format="graphml") # Create new IG graph from file

In [11]:
# generate positions for the nides
pos=graphviz_layout(induced_relabeled,prog='dot')
# add these positions as node features
nx.set_node_attributes(induced_relabeled, "pos", pos)

In [12]:
# calculate general position infos on the whole graph
dmin=1
ncenter=0
for n in pos:
    x,y=pos[n]
    d=(x-0.5)**2+(y-0.5)**2
    if d<dmin:
        ncenter=n
        dmin=d
        
p=nx.single_source_shortest_path_length(induced_relabeled,ncenter)

## Create Edges

In [13]:
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in induced_relabeled.edges():
    x0, y0 = induced_relabeled.node[edge[0]]['pos']
    x1, y1 = induced_relabeled.node[edge[1]]['pos']
    edge_trace['x'] += [x0, x1, None]
    edge_trace['y'] += [y0, y1, None]

node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
        # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
        colorscale='YIGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

for node in induced_relabeled.nodes():
    x, y = induced_relabeled.node[node]['pos']
    node_trace['x'].append(x)
    node_trace['y'].append(y)

## Create Node Colors

In [22]:
#individual node setting
colors = []
descriptions = []
node_sizes = []
sizefactor = 50
for node, adjacencies in enumerate(induced_relabeled.adjacency_list()):
    colors.append(len(adjacencies))
    node_info = 'Node no: '+str(node)+'\nSize: '+str(induced_relabeled.node[node]['size'])+'\n# of connections: '+str(len(adjacencies))
    descriptions.append(node_info)
    node_sizes = list(map(lambda v : (v+1)*sizefactor,normalized_sizes))
    
node_trace['marker']['color'] = colors
node_trace['text'] = descriptions
node_trace['marker']['size'] = node_sizes
    


## Create/Draw Network Graph


In [23]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Network graph',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="...",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

py.iplot(fig, filename='networkx')

c:\python35\lib\site-packages\plotly\tools.py:1416: UserWarning:

Looks like you used a newline character: '\n'.

Plotly uses a subset of HTML escape characters
to do things like newline (<br>), bold (<b></b>),
italics (<i></i>), etc. Your newline characters 
have been converted to '<br>' so they will show 
up right on your Plotly figure!



High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~pcsplotly/0 or inside your plot.ly account where it is named 'networkx'


# igraph

In [24]:
N = len(induced_relabeled.nodes())
layt=induced_ig.layout('kk',dim=3)

In [25]:
Xn=[layt[k][0] for k in range(N)]# x-coordinates of nodes
Yn=[layt[k][1] for k in range(N)]# y-coordinates
Zn=[layt[k][2] for k in range(N)]# z-coordinates
Xe=[]
Ye=[]
Ze=[]
for e in induced_ig.get_edgelist():
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]# x-coordinates of edge ends
    Ye+=[layt[e[0]][1],layt[e[1]][1], None]
    Ze+=[layt[e[0]][2],layt[e[1]][2], None]

In [26]:
trace1=go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=dict(color='rgb(125,125,125)', width=1),
               hoverinfo='none'
               )
trace2=go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='actors',
               marker=dict(symbol='dot',
                             size=node_sizes,
                             color=colors,
                             colorscale='Viridis',
                             line=dict(color='rgb(50,50,50)', width=0.5)
                             ),
               text=descriptions,
               hoverinfo='text'
               )

In [27]:
axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title='')

In [28]:
layout = go.Layout(
         title="Network (3D visualization)",
         width=1000,
         height=1000,
         showlegend=False,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
        ),
     margin=dict(
        t=100
    ),
    hovermode='closest',
    annotations=[
           dict(
           showarrow=False,
            text="...",
            xref='paper',
            yref='paper',
            x=0,
            y=0.1,
            xanchor='left',
            yanchor='bottom',
            font=dict(
            size=14
            )
            )
        ],    )

In [29]:
data=[trace1, trace2]
fig=go.Figure(data=data, layout=layout)

py.iplot(fig, filename='3D')

c:\python35\lib\site-packages\plotly\tools.py:1416: UserWarning:

Looks like you used a newline character: '\n'.

Plotly uses a subset of HTML escape characters
to do things like newline (<br>), bold (<b></b>),
italics (<i></i>), etc. Your newline characters 
have been converted to '<br>' so they will show 
up right on your Plotly figure!

